In [1]:
from sklearn.utils import shuffle
from pyvi import ViTokenizer #For split vietnamese words
import pandas as pd #For reading xlsx file
from gensim.parsing.preprocessing import strip_non_alphanum, strip_multiple_whitespaces,preprocess_string, split_alphanum, strip_short, strip_numeric
import re
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
import string

In [4]:
document = []
label = []
def read_file(file):
    doc = []
    with open(file, 'r', encoding = 'utf-8') as f:
        for line in f:
            doc.append(line)
    doc = shuffle(doc,random_state = 0)
    
    for line in doc:
        document.append(line[7: ])
        if ('non' in line[: 6]):
            label.append(0)
        else:
            label.append(1)

read_file('data_mail.txt')
read_file('data_email.txt')
print(len(document))

391


In [7]:
def raw_text_preprocess(raw):
    raw = raw.lower()
    raw = re.sub(r"http\S+", "URL_Change", raw)
    raw = re.sub(r"https\S+", "URL_Change", raw)
    raw = re.sub(r"(www\S+)|(\S+@\S+)", "URL_Change", raw)
    raw = re.sub(r"([0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9])|([0-9][0-9]/[0-9][0-9])|([0-9][0-9]/[0-9][0-9][0-9][0-9])", "DATE_Change", raw)
    raw = re.sub(r"([0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9])|([0-9][0-9]-[0-9][0-9])|([0-9][0-9]-[0-9][0-9][0-9][0-9])", "DATE_Change", raw)
    raw = re.sub(r"(ngày [0-9][0-9] tháng [0-9][0-9] năm [0-9][0-9][0-9][0-9])|(ngày [0-9][0-9] tháng [0-9][0-9])|(tháng [0-9][0-9] năm [0-9][0-9][0-9][0-9])", "DATE_Change", raw)

    raw = strip_numeric(raw)
    raw = ViTokenizer.tokenize(raw)
    raw = strip_short(raw, minsize=2)
    return raw

In [6]:
#Chia dữ liệu làm 5 đoạn, lấy đoạn thứ i để test và còn lại để train
def divide_fold(i):
    document = [raw_text_preprocess(d) for d in document]
    document_test = document[(i-1)*80:i*80]
    label_test = label[(i-1)*80:i*80]
    document_train = document[:(i-1)*80+1]+document[i*80+1:]
    label_train = label[:(i-1)*80+1]+label[i*80+1:]
    return (document_test,label_test,document_train ,label_train)

In [8]:
def dict_word(document):
    res = {}
    words = {}
    conten = 0
    for doc in document:
#             token = change(doc)
            token = doc.split(" ")
            for tok in token:
                if tok in res.keys():
                    res[tok] = res[tok] + 1
                else:
                    res.update({tok : 1})
            for key, value in res.items():
                if (value >= 3):
                    words.update({key : value})
    words = sorted(words.keys())
    return words

In [9]:
def read_data(document, label, words, fileName):
    word_list = []
    N = len(document)
#     conten = 0
    for doc in document:
#         token = change(doc)
        token = doc.split(" ")
        word = {}
        for tok in token:
            if tok in word.keys():
                word[tok] = word[tok] + 1
            else:
                word.update({tok : 1})
        word_list.append(word)
    data = dat = np.zeros((N, len(words)), dtype = int)
    j = 0
    for word in word_list:
        dat = []
        for key in words:
            if(key in word.keys()):
                dat.append(word[key])
            else:
                dat.append(0)
        data[j, :] = np.array(np.asarray(dat))
        j = j + 1
#     return (np.array(np.asarray(data)), np.array(np.asarray(label)))
    with open(fileName, 'w') as f:
        for i, l in enumerate(label):
            f.write(str(l))
            f.write(" ")
            for num in data[i, :]:
                f.write(str(num))
                f.write(" ")
            f.write('\n')
#     return (data, np.array(np.asarray(label)))

In [15]:
for i in range(1, 6):
    document_test,label_test,document_train ,label_train = divide_fold(i)
    words = dict_word(document_train)
    read_data(document_test,label_test, words, 'test{}.txt'.format(i))
    read_data(document_train,label_train, words, 'train{}.txt'.format(i))

In [12]:
'test{}.txt'.format(12)

'test12.txt'